In [1]:
import pandas as pd

from hardware.opentrons.http_communications import OpentronsAPI

from hardware.cameras import PendantDropCamera

from hardware.sensor.sensor_api import SensorAPI
from hardware.opentrons.formulater import Formulater

from hardware.opentrons.configuration import Configuration
from analysis.plots import Plotter

from utils.logger import Logger
from utils.load_save_functions import load_settings
from utils.search_containers import get_well_id_concentration
from analysis.active_learning import ActiveLearner
from analysis.models import szyszkowski_model

from hardware.opentrons.droplet_manager import DropletManager


opentrons_api = OpentronsAPI()

opentrons_api.initialise()

config = Configuration(http_api=opentrons_api)

labware = config.load_labware()

containers = config.load_containers()

pipettes = config.load_pipettes()

left_pipette = pipettes["left"]

right_pipette = pipettes["right"]

pendant_drop_camera = PendantDropCamera()
sensor_api = SensorAPI()
plotter = Plotter()
settings = load_settings()
logger = Logger(
    name="protocol",
    file_path=f'experiments/{settings["EXPERIMENT_NAME"]}/meta_data',
)
# formulater = Formulater(
#     left_pipette=left_pipette,
#     right_pipette=right_pipette,
#     containers=containers,
#     logger=logger
# )
droplet_manager = DropletManager(
    left_pipette=left_pipette,
    containers=containers,
    pendant_drop_camera=pendant_drop_camera,
    opentrons_api=opentrons_api,
    plotter=plotter,
    logger=logger

)

c:\Users\pimda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-21 13:52:25,775 - INFO - Protocol already uploaded, using existing protocol.
2025-03-21 13:52:32,858 - INFO - Run created succesfully (ID: 66de1fca-bf77-4dfb-85e2-76cab28d08b8).
2025-03-21 13:52:35,341 - INFO - All custom labware definitions added.
2025-03-21 13:52:36,175 - INFO - Labware loaded successfully.
2025-03-21 13:52:36,224 - INFO - Containers loaded successfully.
2025-03-21 13:52:40,310 - INFO - Pipettes loaded successfully.


In [2]:
opentrons_api.home()

2025-03-21 13:52:59,415 - INFO - Robot homed.


In [8]:
left_pipette.has_needle

True

In [3]:
left_pipette.pick_up_needle()

2025-03-21 13:47:46,058 - INFO - Picked up needle.


In [4]:
left_pipette.wash()

2025-03-21 13:47:46,363 - INFO - Starting washing needle.
2025-03-21 13:47:58,474 - INFO - Done with mixing in 3C4 with order after, with volume 20 uL, repeated 4 times
2025-03-21 13:47:59,213 - INFO - Drying needle.
2025-03-21 13:48:01,327 - INFO - Aspirated 20 uL from 3C5 with left pipette.
2025-03-21 13:48:02,573 - INFO - Dispensed 20 uL into well 3C5 with left pipette.
2025-03-21 13:48:03,908 - INFO - Aspirated 20 uL from 3C5 with left pipette.
2025-03-21 13:48:05,141 - INFO - Dispensed 20 uL into well 3C5 with left pipette.
2025-03-21 13:48:06,474 - INFO - Aspirated 20 uL from 3C5 with left pipette.
2025-03-21 13:48:07,724 - INFO - Dispensed 20 uL into well 3C5 with left pipette.
2025-03-21 13:48:09,058 - INFO - Aspirated 20 uL from 3C5 with left pipette.
2025-03-21 13:48:10,323 - INFO - Dispensed 20 uL into well 3C5 with left pipette.
2025-03-21 13:48:11,657 - INFO - Aspirated 20 uL from 3C5 with left pipette.
2025-03-21 13:48:12,891 - INFO - Dispensed 20 uL into well 3C5 with le

In [4]:
left_pipette.return_needle()

2025-03-21 14:05:29,157 - INFO - Returned needle.


In [3]:
drop_parameters = {"drop_volume": 11, "max_measure_time": 60, "flow_rate": 1}
droplet_manager.measure_pendant_drop(
    source=containers["7A3"],
    drop_parameters=drop_parameters,
)

2025-03-21 13:53:05,668 - INFO - Picked up needle.
2025-03-21 13:53:05,673 - INFO - Start measurment of pendant drop of 7A3 with drop volume 11.0 uL and drop count 1.
2025-03-21 13:53:09,258 - WARNING - Aspiration volume is larger than container volume!
2025-03-21 13:53:09,258 - INFO - Aspirated 17 uL from 7A3 with left pipette.
2025-03-21 13:53:13,190 - INFO - Air gap of 3 uL performed in left pipette.
2025-03-21 13:53:17,380 - INFO - Tip/needle cleaned on sponge.
2025-03-21 13:53:22,308 - INFO - Air gap of 3 uL removed in left pipette.
2025-03-21 13:53:22,311 - INFO - Dispensing pendant drop.
2025-03-21 13:53:34,307 - INFO - Camera: start measuring 7A3.
2025-03-21 13:54:34,792 - INFO - Camera: stopped measurement
2025-03-21 13:54:35,757 - INFO - Re-aspirated the pendant drop into the tip.
2025-03-21 13:54:39,220 - INFO - Container: dispensed 17 uL into this container from source 7A3 containing n.a. mM empty.
2025-03-21 13:54:39,220 - INFO - Dispensed 17 uL into well 7A3 with left pip

([[0.918836, 55.91245560359556],
  [1.30344, 59.22508600314799],
  [1.513943, 51.07799170829974],
  [1.738913, 56.54614836861572],
  [1.957913, 58.541653264021164],
  [2.234347, 49.11035735595096],
  [2.448368, 51.34486006528136],
  [2.653133, 54.95773884940362],
  [2.854932, 51.95161025642534],
  [3.068989, 50.49876459564242],
  [3.300521, 53.11027605285815],
  [3.502972, 51.36167607162295],
  [3.722929, 53.409666315385145],
  [3.932181, 51.91981899450824],
  [4.15786, 51.043571917740564],
  [4.433699, 51.3282697515545],
  [4.665193, 53.39663425764229],
  [4.897801, 52.195619959549155],
  [5.116672, 52.48922943254993],
  [5.358771, 52.47500262912413],
  [5.611824, 52.48922943254993],
  [5.8391, 53.38385373196253],
  [6.045543, 53.070219829844135],
  [6.250194, 53.37132635487388],
  [6.453944, 53.975007940311414],
  [6.706549, 51.8590410697688],
  [6.934217, 52.44727986066948],
  [7.142525, 52.152015998949054],
  [7.355875, 53.34074782479939],
  [7.57008, 51.84443945587203],
  [7.80114

In [19]:
left_pipette.aspirate(
    volume=17,
    source=containers["7A5"]
)

2025-03-21 12:01:21,565 - WARNING - Aspiration volume is larger than container volume!
2025-03-21 12:01:21,565 - INFO - Aspirated 17 uL from 7A5 with left pipette.


In [20]:
left_pipette.air_gap(air_volume=3)

2025-03-21 12:01:28,994 - INFO - Air gap of 3 uL performed in left pipette.


In [21]:
left_pipette.clean_on_sponge()

2025-03-21 12:01:33,026 - INFO - Tip/needle cleaned on sponge.


In [22]:
left_pipette.remove_air_gap(at_drop_stage=True)

2025-03-21 12:01:39,057 - INFO - Air gap of 3 uL removed in left pipette.


In [24]:
left_pipette.dispense(
    volume=1,
    destination=containers["drop_stage"],
    flow_rate=1,
    depth_offset=-23.4
)

2025-03-21 12:02:10,529 - INFO - Dispensed 1 uL into well 10A1 with left pipette.


In [25]:
left_pipette.dispense(volume=left_pipette.volume, destination=containers["7A5"])

2025-03-21 12:02:23,824 - INFO - Container: dispensed 10.0 uL into this container from source 7A5 containing n.a. mM empty.
2025-03-21 12:02:23,824 - INFO - Dispensed 10.0 uL into well 7A5 with left pipette.


In [26]:
left_pipette.return_needle()

2025-03-21 12:02:38,644 - INFO - Returned needle.


In [6]:
offset = {
    'x': 0,
    'y':0 ,
    'z':10,
}
left_pipette.move_to_well(container=containers["3C4"], offset=offset)

In [14]:
offset = {
    "x": 0,
    "y": 1,
    "z": -30,
}
left_pipette.move_to_well(container=containers["3C5"], offset=offset)

In [5]:
left_pipette.return_needle()

2025-03-20 17:49:29,574 - INFO - Returned needle.
